In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

## Table of content: <a class="anchor" id="top" name="top"></a>
- [Introduction](#intro)
- [Data preparation](#data-preparation)
- [Model selection, training and evaluation](#model-training)
- [Next steps](#next-steps)

# I - Introduction <a class="anchor" id="intro" name="intro"></a>

[-> table of content](#top)

The goal is to predict the daily sales based on the available information (date, product, ...) for different discounts.

I decided to train a model to predict the sales for any possible discount. The discount is thus considered as an input feature for the model. If we only wish to predict the sales for a small set of discount values (e.g -10%, -20%, -30%, ...), another approach could be to train a different model for each discount value but that would require more resources.

I also assume that all the sales information is present in the data, i.e. the sales are reported even if they are null. And, if no sales are reported for a given product at a given date, it means that none of them were for sale. Otherwise, we should add the missing data.

In the following, I first prepare the data using [pandas](https://pandas.pydata.org/), then use the [scikit-learn](https://scikit-learn.org/stable/) library to try different set of parameters to train an [XGBoost](https://xgboost.readthedocs.io/en/stable/) model. I chose XGBoost because it is a versatile model that performs very well on many tasks.

# II - Data preparation <a class="anchor" id="data-preparation" name="data-preparation"></a>

[-> table of content](#top)

### 1. Loading and cleaning

I found some data with a negative number of sales, some undiscounted sales with a price of 0 and some cases where the discounted price was larger than the undiscounted price. I decided to discard them as they were very few.

In [2]:
DATA_FILE = "data/test_technique ML eng - data.csv"

df = pd.read_csv(DATA_FILE, sep=';', dtype={'ean': 'str'}, parse_dates=['date'], date_parser=lambda x: datetime.strptime(x, "%Y-%m-%d"))
print("Raw data count:", len(df))

# remove data with negative sales
df.drop(df[(df['nb_vente'] < 0) | (df['nb_vente_kcf'] < 0)].index, inplace=True)

# remove data where pv=0
df.drop(df[df['pv'] == 0 ].index, inplace=True)

# remove data where pv_kcf >= pv
df.drop(df[df['pv'] <= df['pv_kcf'] ].index, inplace=True)

print("Cleaned data count:", len(df))

Raw data count: 138882
Cleaned data count: 138724


## 2. Extracting discount data

Each row can contain some data about discounted and undiscounted sales. In this part, I split the rows to get a single price and sales count per row. I also added the discount percentage as a new feature as the discount could have a psychological effect on the customers and thus should be taken into account when trying to predict the sales.

In [3]:
METADATA_COLS = ['date', 'ean', 'qt_casse']

In [4]:
def extract_undiscounted_data(df):

    undiscounted_df = df.loc[:, METADATA_COLS + ['pv', 'nb_vente']]
    undiscounted_df.rename(columns={'pv': 'price', 'nb_vente': 'sales_count'}, inplace=True)
    undiscounted_df['disc_percentage'] = 0

    return undiscounted_df

def extract_discounted_data(df):

    masked_df = df[df['pv_kcf'] > 0]
    discounted_df = masked_df.loc[:, METADATA_COLS + ['pv_kcf', 'nb_vente_kcf']]
    discounted_df.rename(columns={'pv_kcf': 'price', 'nb_vente_kcf': 'sales_count'}, inplace=True)
    discounted_df['disc_percentage'] = masked_df['pv_kcf'] / masked_df['pv']

    return discounted_df

In [5]:
undiscounted_df = extract_undiscounted_data(df)
print("Undiscounted data count:", len(undiscounted_df))

discounted_df = extract_discounted_data(df)
print("Discounted data count:", len(discounted_df))

all_discounts_df = pd.concat([undiscounted_df, discounted_df])
print("All discounts data count:", len(all_discounts_df))

all_discounts_df.head()

Undiscounted data count: 138724
Discounted data count: 9405
All discounts data count: 148129


,date,ean,qt_casse,price,sales_count,disc_percentage
0,2022-04-01,40846.0,0.0,1.49,2.0,0.0
1,2022-04-01,3760253432320.0,0.0,5.65,1.0,0.0
2,2022-04-01,3222473152865.0,0.0,5.20,2.0,0.0
3,2022-04-01,3222473233687.0,0.0,3.95,6.0,0.0
4,2022-04-01,3760253430623.0,0.0,8.50,1.0,0.0


As we can see, the amount of data about discounted sales is much lower than the amount of data about undiscounted sales. This could affect the model ability to predict discounted sales.

## 3. Generating date features

The data span 5 months of 2022. Thus, we don't have enough information to change the prediction based on the year or month. I only keep the day of the week (monday, tuesday, ...) and weither it's the beginning, middle or end of the month.

In [6]:
def generate_date_features(df):

    def get_month_part(x):
        if x.day <= 10:
            return 0
        elif x.day <= 20:
            return 1
        else:
            return 2

    df['month_part'] = df['date'].map(lambda x: get_month_part(x))
    df['weekday'] = df['date'].map(lambda x: x.weekday())

In [7]:
generate_date_features(all_discounts_df)

In [8]:
all_discounts_df.head()

,date,ean,qt_casse,price,sales_count,disc_percentage,month_part,weekday
0,2022-04-01,40846.0,0.0,1.49,2.0,0.0,0,4
1,2022-04-01,3760253432320.0,0.0,5.65,1.0,0.0,0,4
2,2022-04-01,3222473152865.0,0.0,5.20,2.0,0.0,0,4
3,2022-04-01,3222473233687.0,0.0,3.95,6.0,0.0,0,4
4,2022-04-01,3760253430623.0,0.0,8.50,1.0,0.0,0,4


## 4. Generating aggregated features based on product identifiers

There are around 2.5k different products in the data. We cannot give their ids directly to the model and a simple [one hot encoding](https://en.wikipedia.org/wiki/One-hot) would require us to add 2.5k new columns. A good approach is to use this information to generate aggregated features instead. 

Thus, for each product I compute:
- the average undiscounted price
- the average trashed quantity
- the average number of days between undiscounted sales

I decided not to take discounted sales information into account for aggregated features as they might add a lot of variability.

Also, a critical point is to avoid a so called "[information leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning))" by using information of future sales to predict the current ones as this information would not be available in real case scenarios. Thus, for each record, we can only aggregate information from the previous records.

In [9]:
def generate_aggregated_features(df):

    df.loc[:, ('agg_price', 'agg_qt_casse', 'agg_sales_count')] = np.nan

    def add_agg_features(group):
        # get undiscounted rows
        group_undisc = group[group['disc_percentage']==0].sort_values(by='date', ascending=True)

        for x in group.itertuples():

            # get previous sales
            prev_undisc = group_undisc[group_undisc['date'] < x.date]

            agg_days_bt_sales = prev_undisc[prev_undisc['sales_count'] > 0]['date'].diff().map(lambda x: x.days).mean()
            agg_price = prev_undisc['price'].mean()
            agg_qt_casse = prev_undisc['qt_casse'].mean()
            agg_sales_count = prev_undisc['sales_count'].mean()

            df.loc[x.Index, 'agg_days_bt_sales'] = agg_days_bt_sales
            df.loc[x.Index, 'agg_price'] = agg_price
            df.loc[x.Index, 'agg_qt_casse'] = agg_qt_casse
            df.loc[x.Index, 'agg_sales_count'] = agg_sales_count
    
    df.groupby('ean').apply(add_agg_features)

In [10]:
# this can take a few minutes, do it once then use the saved result directly.
generate_aggregated_features(all_discounts_df)

# saving the final dataframe
DF_FILE = 'data/all_discounts_df.csv'
all_discounts_df.to_csv(DF_FILE, index=False)

all_discounts_df.tail()

,date,ean,qt_casse,price,sales_count,disc_percentage,month_part,weekday,agg_price,agg_qt_casse,agg_sales_count,agg_days_bt_sales
138773,2022-08-31,3222476292353.0,0.0,0.83,3.0,0.664000,2,2,1.203806,0.160000,1.733333,1.985915
138796,2022-08-31,3222474499242.0,0.0,2.21,1.0,0.789286,2,2,2.864117,0.089286,1.625000,2.882353
138831,2022-08-31,3180950007292.0,0.0,2.04,1.0,0.660194,2,2,2.739446,0.049383,1.802469,1.911392
138869,2022-08-31,3222477010390.0,0.0,2.08,1.0,0.658228,2,2,2.959337,0.293333,2.933333,2.083333
138879,2022-08-31,3222474698461.0,0.0,3.27,1.0,0.642436,2,2,4.760699,0.224138,2.206897,2.830189


# III - Model selection, training and evaluation <a class="anchor" id="model-training" name="model-training"></a>

[-> table of content](#top)

In [2]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from collections import OrderedDict
import random as rd

## 1. Generating the features and labels

In this part, we keep the usefull columns for our model and generate dummy columns using one hot encoding for month parts and the days of the week.

In [3]:
FEATURES_COLS = ['price', 'disc_percentage', 'month_part', 'weekday', 'agg_price', 'agg_qt_casse', 'agg_sales_count', "agg_days_bt_sales"]

In [4]:
DF_FILE = 'data/all_discounts_df.csv'
all_discounts_df = pd.read_csv(DF_FILE).sort_values(by='date') # ensure sorted by date
X = all_discounts_df.loc[:, FEATURES_COLS]
X = pd.get_dummies(X, prefix='month_parts', columns=['month_part'])
X = pd.get_dummies(X, prefix='weekday', columns=['weekday'])
y = all_discounts_df['sales_count']

X.tail()

,price,disc_percentage,agg_price,agg_qt_casse,agg_sales_count,agg_days_bt_sales,month_parts_0,month_parts_1,month_parts_2,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
138215,3.55,0.000000,3.396154,0.000000,1.692308,3.894737,0,0,1,0,0,1,0,0,0,0
138216,5.85,0.000000,4.991727,0.104478,1.895522,2.426230,0,0,1,0,0,1,0,0,0,0
138217,3.99,0.000000,3.227279,0.363636,1.795455,3.609756,0,0,1,0,0,1,0,0,0,0
138147,8.70,0.000000,8.100000,0.000000,2.260870,4.727273,0,0,1,0,0,1,0,0,0,0
148128,3.27,0.642436,4.760699,0.224138,2.206897,2.830189,0,0,1,0,0,1,0,0,0,0


## 2. Model selection

We need to select good hyperparameters for our models (meta parameters that impact the training and performance of the model). To do that, we usually train the same model on the same data with different set of parameters. Vairous approaches exist, in this case we use a randomized search in which we pick hyperparameters randomly among a set of possible values. It is a simple approach that allows us to test different settings easily.

To evaluate the model, we cannot use a standard [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)). Indeed, here too we should carefully avoid "information leakage". Thus, we instead use scikit-learn [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) to ensure that we only test the model on future sales (instead of potentially training on future sales to predict the current ones).

The metric used to train the model is a standard squared error. I chose this metric rather than the absolute error as I assumed that 1 big sales predictions mistakes is worse than a few small ones.

In [5]:

params_distributions = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0, 0.1, 10],
   'min_child_weight':[1, 5, 10],
   'learning_rate':[0.01, 0.1, 0.2],
   'max_depth': range(3,10),
   'reg_alpha':[1e-2, 1e-1,  1, 10],
   'reg_lambda':[1e-2, 1e-1, 1, 10],
   'subsample':[0.6,0.95],
   "objective": ["reg:squarederror"],
   "n_estimators": [200, 300, 500]
}

                    
xgb_model = xgb.XGBRegressor(seed=27)
cv = TimeSeriesSplit(n_splits=3).split(X)

rsearch = RandomizedSearchCV(
   estimator = xgb_model,
   param_distributions = params_distributions,
   n_iter = 10,
   verbose = 10,
   cv = cv,
   refit = False,
   scoring = 'neg_root_mean_squared_error')

rsearch.fit(X, y)

print('best score')
print (rsearch.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START colsample_bytree=0.8, gamma=10, learning_rate=0.1, max_depth=7, min_child_weight=10, n_estimators=200, objective=reg:squarederror, reg_alpha=0.01, reg_lambda=10, subsample=0.6
[CV 1/3; 1/10] END colsample_bytree=0.8, gamma=10, learning_rate=0.1, max_depth=7, min_child_weight=10, n_estimators=200, objective=reg:squarederror, reg_alpha=0.01, reg_lambda=10, subsample=0.6;, score=-2.492 total time=   6.2s
[CV 2/3; 1/10] START colsample_bytree=0.8, gamma=10, learning_rate=0.1, max_depth=7, min_child_weight=10, n_estimators=200, objective=reg:squarederror, reg_alpha=0.01, reg_lambda=10, subsample=0.6
[CV 2/3; 1/10] END colsample_bytree=0.8, gamma=10, learning_rate=0.1, max_depth=7, min_child_weight=10, n_estimators=200, objective=reg:squarederror, reg_alpha=0.01, reg_lambda=10, subsample=0.6;, score=-2.690 total time=  16.2s
[CV 3/3; 1/10] START colsample_bytree=0.8, gamma=10, learning_rate=0.1, max_depth=7, mi

## 3. Model training and evaluation

In this part, we train the model on 2/3 of the dataset and evaluate on the last one to be able to analyse a little bit more the results.

In [6]:
model = xgb.XGBRegressor(**rsearch.best_params_, seed=27)

split_id = 2 * len(X) // 3
X_train, y_train = X.iloc[:split_id], y.iloc[:split_id]
X_test, y_test = X.iloc[split_id:], y.iloc[split_id:]

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=10, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=10,
             missing=nan, monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=27,
             reg_alpha=0.01, reg_lambda=10, ...)

In [7]:
print("Feature importance: (feature, usage)")
OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

Feature importance: (feature, usage)


OrderedDict([('agg_sales_count', 2265.0),
             ('price', 2170.0),
             ('agg_price', 2122.0),
             ('agg_days_bt_sales', 1485.0),
             ('agg_qt_casse', 787.0),
             ('disc_percentage', 297.0),
             ('weekday_4', 250.0),
             ('weekday_5', 246.0),
             ('month_parts_0', 150.0),
             ('month_parts_2', 148.0),
             ('weekday_6', 146.0),
             ('month_parts_1', 123.0),
             ('weekday_1', 92.0),
             ('weekday_0', 80.0),
             ('weekday_3', 64.0),
             ('weekday_2', 61.0)])

We can see that the aggregated price feature has been used 11816 times by the model and the information about some days of the week only a few hundreds time. The aggregated features are thus the most important for the model. 

Interestingly, we can see that the model seems to base its decision on wether it's friday, saturday or sunday (weekday_4, weekday_5, weekday_6) more than the other days.

Next, we will compute the root mean squared error and mean absolute error on the test set and check a few predictions by hand.

In [8]:
pred = model.predict(X_test)
err = pred-y_test
print(f"Test root mean squared error (RMSE): {np.sqrt(err.map(lambda x: x**2).mean()):.4}",)
print(f"Test mean absolute error (MAE): {err.map(abs).mean():.4}")

discounted_mask = X_test['disc_percentage'] > 0
print(f"\nTest RMSE (undiscounted sales only): {np.sqrt(err[~discounted_mask].map(lambda x: x**2).mean()):.4}")
print(f"Test MAE (undiscounted sales only): {err[~discounted_mask].map(abs).mean():.4}")
print(f"\nTest RMSE (discounted sales only): {np.sqrt(err[discounted_mask].map(lambda x: x**2).mean()):.4}")
print(f"Test MAE (discounted sales only): {err[discounted_mask].map(abs).mean():.4}")

Test root mean squared error (RMSE): 2.61
Test mean absolute error (MAE): 1.567

Test RMSE (undiscounted sales only): 2.611
Test MAE (undiscounted sales only): 1.555

Test RMSE (discounted sales only): 2.596
Test MAE (discounted sales only): 1.727


The RMSE cannot easily be interpreted but the MAE is the error that the model is doing in average for each sale prediction. 

We can see that the RMSE is the same on discounted and undiscounted sales but the model has a higher MAE on discounted sales. If this is a problem we could add weights to the training data so that the model focuses more on discounted sales.

In [19]:
print("Sample of predictions:")
for _ in range(15):
    i = rd.randint(0, len(X_test))
    ind = X_test.index[i]
    rpred = round(pred[i])
    true = y_test.iloc[i]
    print(all_discounts_df.loc[ind, 'date'], all_discounts_df.loc[ind, 'ean'],  f"prediction: {rpred} true value: {true}")

Sample of predictions:
2022-08-24 7613036855693.0 prediction: 1 true value: 2.0
2022-07-27 3222477061262.0 prediction: 4 true value: 4.0
2022-07-13 3248832960025.0 prediction: 2 true value: 2.0
2022-08-22 3242272348252.0 prediction: 2 true value: 1.0
2022-08-29 3222474215545.0 prediction: 2 true value: 3.0
2022-08-11 3302746583029.0 prediction: 2 true value: 1.0
2022-07-23 3222477544284.0 prediction: 2 true value: 1.0
2022-08-24 3154230800514.0 prediction: 2 true value: 1.0
2022-08-08 3095756221011.0 prediction: 2 true value: 3.0
2022-07-30 3180940084371.0 prediction: 7 true value: 6.0
2022-08-06 3095756312016.0 prediction: 4 true value: 1.0
2022-08-22 3095754137017.0 prediction: 1 true value: 1.0
2022-08-30 3436590082268.0 prediction: 3 true value: 2.0
2022-08-18 3222472606253.0 prediction: 4 true value: 3.0
2022-07-09 7613034423146.0 prediction: 3 true value: 1.0


# IV - Next steps <a class="anchor" id="next-steps" name="next-steps"></a>

[-> table of content](#top)

## 1. Deployment

Such a model can easily be deployed as an HTTP API (e.g. using [FastAPI](https://fastapi.tiangolo.com/)) with a `/predict` POST endpoint to predict daily sales one by one or by batches.

The main issue is the footprint of the model as ML models can become quite big very fast. We should find the right tradeoff between having a good accuracy, a good latency / throughput, and not consuming to many resources.

## 2. Model improvement

To improve the model, one could:
- try to find better features and remove the ones that are not usefull
- try more hyperparameters for the model
- try different models than XGBoost
- try to frame the problem in a different way e.g. predicting how much time it takes to sell a product with a given discount rather than predicting the daily sales.
- try to add sample weights on the data to minimize imbalances
- try to gather more data (quantity and features, e.g. stores location)

## 3. Model monitoring

A simple way to test whether our model improves or not is to compare the prediction of the model based on real world data. If we see a drop in the performance, we can simply retrain the model.

However, we would be in a typical "Exploration-Exploitation" dilemma. Indeed, each day we can only apply 1 discount value for a given product. Therefore, we can only get the true number of sales for a single discount value. If we always choose the best discount (exploitation), the model could never learn (or forget) how to predict the sales in other scenarios and the predictions would likely not be optimal.

If we take suboptimal decisions from time to time to gather data (exploration), we would improve the ability of the model to handle various situations but we would pay the price of the suboptimal discounts. Thus, a good tradeoff has to be found.

Additionally, we could monitor other statistics of the data (e.g. average sales, prices etc.) to detect data distribution [drifts](https://en.wikipedia.org/wiki/Drift_(data_science)), i.e. when the real world data becomes too different from the data the model has been trained on. In this case, the model is likely to become obsolete and needs to be trained again.
